## Foundry Agent with Code Interpreter Capability

### Installing Required Libraries

In [ ]:

%pip install azure-ai-projects==2.0.0b2 
%pip install openai==1.109.1 
%pip install python-dotenv
%pip install azure-identity

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, CodeInterpreterTool, CodeInterpreterToolAuto

load_dotenv(dotenv_path=os.path.join(os.pardir, ".env"))

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

### Setting Up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### CSV File Upload

In [ ]:
# Upload the CSV file for the code interpreter to use
file = openai_client.files.create(purpose="assistants", file=open("./electronics_products.csv", "rb"))
print(f"File uploaded (id: {file.id})")

### Creating Agent with the Code Interpreter Tool

In [ ]:
agent_name = "code-interpreter-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful AI Assistant with code interpreter capabilities.",
        tools = [
            CodeInterpreterTool(
                container = CodeInterpreterToolAuto(
                    file_ids = [file.id]
                )
            )
        ]
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Calling Our Agent

In [ ]:
response = openai_client.responses.create(
    conversation = conversation.id,
    input = "Could you please create a column chart with products on the x-axis and their respective prices on the y-axis?",
    extra_body = {
        "agent": {
            "name": agent.name,
            "type": "agent_reference"
        }
    }
)

print(f"Response completed with id: {response.id}")

### Extracting file information from response annotations

In [ ]:
file_id = ""
filename = ""
container_id = ""

# Get the last message which should contain file citations
last_message = response.output[-1]  # ResponseOutputMessage
if last_message.type == "message":
        # Get the last content item (contains the file annotations)
        text_content = last_message.content[-1]  # ResponseOutputText
        if text_content.type == "output_text":
            # Get the last annotation (most recent file)
            if text_content.annotations:
                file_citation = text_content.annotations[-1]  # AnnotationContainerFileCitation
                if file_citation.type == "container_file_citation":
                    file_id = file_citation.file_id
                    filename = file_citation.filename
                    container_id = file_citation.container_id
                    print(f"Found generated file: {filename} (ID: {file_id})")

# Download the generated file if available
if file_id and filename:
        file_content = openai_client.containers.files.content.retrieve(file_id=file_id, container_id=container_id)
        with open(filename, "wb") as f:
            f.write(file_content.read())
            print(f"File {filename} downloaded successfully.")
        print(f"File ready for download: {filename}")
else:
        print("No file generated in response")